# SVC, Random Forest, and XBoost

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import pickle
from datetime import datetime, timedelta 

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC, SVC



import warnings
warnings.filterwarnings('ignore')

from imblearn.under_sampling import NearMiss

ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (/Users/ryan/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/__init__.py)

In [2]:
with open('pickles/df.pkl', 'rb') as f:
    df = pickle.load(f)

In [3]:
df.shape

(52206, 21)

In [4]:
df['STAT_CAUSE_DESCR'].value_counts()

Lightning         19825
Negligence        13595
Infrastructure     9829
Arson              8957
Name: STAT_CAUSE_DESCR, dtype: int64

In [5]:
lb = LabelEncoder()
y = lb.fit_transform(df['STAT_CAUSE_DESCR'])

In [6]:
num_data = df.select_dtypes('number')

In [7]:
cat_cols = df.select_dtypes('object').columns
cat_cols

Index(['STAT_CAUSE_DESCR', 'STATE', 'SOURCE_SYSTEM_TYPE', 'FIRE_SIZE_CLASS',
       'SOURCE_REPORTING_UNIT_NAME'],
      dtype='object')

In [8]:
X = df.drop(columns = ['STAT_CAUSE_DESCR', 'STATE', 'SOURCE_REPORTING_UNIT_NAME'])

In [9]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = 0.4,
                                                    random_state = 42, stratify = y)

In [10]:
#Fitting LOGISTIC Regression
log = LogisticRegression()
column_trans = make_column_transformer(
(OneHotEncoder(), ['SOURCE_SYSTEM_TYPE', 'FIRE_SIZE_CLASS']), 
remainder = 'passthrough')
column_trans.fit_transform(X_train, y_train)
pipe = make_pipeline(column_trans,log)
print(f'Train Score: {cross_val_score(pipe,X_train,y_train, cv = 5)}')
print(f'Test Score:{cross_val_score(pipe, X_test, y_test, cv = 5)}')

/Users/ryan/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ryan/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

Train Score: [0.54828412 0.54317638 0.54317638        nan 0.53799489]


/Users/ryan/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ryan/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

Test Score:[0.53794589 0.55925305 0.53291836 0.55555556 0.5414272 ]


/Users/ryan/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
pipe.fit(X_train, y_train)
print(pipe.score(X_train, y_train))
print(pipe.score(X_test, y_test))

0.5457970181655653
0.5458028061102332


/Users/ryan/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
num_data.columns

Index(['DISCOVERY_DOY', 'duration', 'LATITUDE', 'LONGITUDE', 'FIRE_YEAR',
       'FIRE_SIZE', 'elevation', 'bdod', 'cec', 'cfvo', 'clay', 'nitrogen',
       'phh2o', 'sand', 'silt', 'soc'],
      dtype='object')

In [13]:
X = df.drop(columns = ['STAT_CAUSE_DESCR', 'STATE', 'SOURCE_REPORTING_UNIT_NAME', 
                      'DISCOVERY_DOY', 'FIRE_YEAR'])

In [14]:
new_df = df.drop(columns = ['STAT_CAUSE_DESCR', 'STATE', 'SOURCE_REPORTING_UNIT_NAME', 
                      'DISCOVERY_DOY', 'FIRE_YEAR'])

In [15]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = 0.3,
                                                    random_state = 42, stratify = y)

In [16]:
#Fitting Linear Regression with scaling
log = LogisticRegression()
column_trans = make_column_transformer(
(StandardScaler(), ['duration', 'LATITUDE', 'LONGITUDE',
                       'FIRE_SIZE', 'elevation', 'bdod', 'cec', 'cfvo', 'clay', 'nitrogen',
                   'phh2o', 'sand', 'silt', 'soc']),
(OneHotEncoder(), ['SOURCE_SYSTEM_TYPE', 'FIRE_SIZE_CLASS']), 
remainder = 'passthrough')
column_trans.fit_transform(X_train, y_train)
pipe = make_pipeline(column_trans,log)

print(f'Train Score: {cross_val_score(pipe,X_train,y_train, cv = 5)}')
print(f'Test Score:{cross_val_score(pipe, X_test, y_test, cv = 5)}')

NameError: name 'StandardScaler' is not defined

In [17]:
pipe.fit(X_train, y_train)

/Users/ryan/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['SOURCE_SYSTEM_TYPE',
                                                   'FIRE_SIZE_CLASS'])])),
                ('logisticregression', LogisticRegression())])

In [18]:
y_preds = pipe.predict(X_test)

In [19]:
df['STAT_CAUSE_DESCR'].unique()

array(['Lightning', 'Negligence', 'Infrastructure', 'Arson'], dtype=object)

In [ ]:
plot_confusion_matrix(pipe, X_test, y_test, cmap='Blues', display_labels = ['Miscellaneous', 'Lightning', 'Negligence', 'Equipment Use',
       'Children', 'Arson', 'Infrastructure'] )
plt.xticks(rotation = 90);

In [ ]:
df_dummies = pd.get_dummies(new_df, drop_first = True)

In [ ]:
with open('./datasets/df_final.pkl', 'rb') as f:
    df_final = pickle.load(f)

In [ ]:
df_final.head()

In [ ]:
df_final['STAT_CAUSE_DESCR'].value_counts()

In [ ]:
lb = LabelEncoder()
df_final['status_cause_labels'] = lb.fit_transform(df_final['STAT_CAUSE_DESCR'])

In [ ]:
y = df_final['status_cause_labels']

In [ ]:
df_final['status_cause_labels'].value_counts()

In [ ]:
cf_labels = ['Arson', 'Infrastructure', 'Negligence', 'Negligence']

In [ ]:
df_final.drop(columns = ['STAT_CAUSE_DESCR', 'STATE', 'SOURCE_REPORTING_UNIT_NAME', 
                      'DISCOVERY_DOY', 'FIRE_YEAR', 'FIRE_SIZE_CLASS'], axis = 1, inplace = True)

In [ ]:
df_final.head()

In [ ]:
final_dummies = pd.get_dummies(df_final, drop_first = True)

In [ ]:
final_dummies.head()

In [ ]:
final_dummies.drop(columns = 'status_cause_labels', inplace = True)

In [ ]:
X = final_dummies.copy()

In [ ]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = 0.3,
                                                    random_state = 42, stratify = y)

In [ ]:
rf_params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 1, 2, 3],
}

In [ ]:
rf = RandomForestClassifier()

In [ ]:
gs = GridSearchCV(rf,
                  param_grid = rf_params, # what object are we optimizing?
                  # what parameters values are we searching?
                  cv = 5)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

In [ ]:
plot_confusion_matrix(gs, X_test, y_test, cmap='Blues', display_labels = cf_labels )
plt.xticks(rotation = 90);

In [ ]:
final_dummies_1 = final_dummies.drop(columns = ['FIRE_SIZE', 'bdod', 'sand', 'silt', 'soc'])

In [ ]:
X = final_dummies_1.copy()

In [ ]:
X.shape

In [ ]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = 0.3,
                                                    random_state = 42, stratify = y)

In [ ]:
#XGBoost Classifier
xgb = XGBClassifier()

In [ ]:
#Defining Parameters
xgb_params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 1]
}

In [ ]:
#GridsearchCV
gs_xgb = GridSearchCV(xgb, # what object are we optimizing?
                  param_grid = xgb_params,# what parameters values are we searching?
                  cv = 5)

In [ ]:
#Fitting XGBoost
gs_xgb.fit(X_train, y_train)

In [ ]:
#Best Score and parametres with XGBoost Model
print(gs_xgb.best_score_)
print(gs_xgb.best_params_)

In [ ]:
plot_confusion_matrix(gs_xgb, X_test, y_test, cmap='Greens', display_labels = cf_labels)
plt.xticks(rotation = 60);

In [ ]:
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

In [ ]:
pgrid = {"C": np.linspace(0.0001, 2, 6)}

In [ ]:
# (THREAD) Instantiate and fit a gridsearch model for this SVC!
svc = LinearSVC(max_iter= 1000)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

gcv = GridSearchCV(svc, pgrid, cv=cv)
gcv.fit(X_train_ss, y_train);

In [ ]:
gcv.best_score_

In [ ]:
# Save cv results as a DataFrame
df_svc = pd.DataFrame(gcv.cv_results_)

# Visualize how different values of C affect accuracy
plt.plot("param_C", "mean_test_score", data = df_svc);